In [ ]:
!nvidia-smi

In [ ]:
# Import your libraries
from PIL import Image
from pathlib import Path
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
path = Path('/kaggle/input/glass-bangle-defect-detection-classification')
images_list = list(path.glob('**/*.jpg'))

img_cv = cv2.imread(str(images_list[0]))

print(type(img_cv), img_cv.shape)
# Resize img_cv -> 224 x 224
img_cv_resized = cv2.resize(img_cv, (224, 224))

# Change the color channel
img_cv_resized = cv2.cvtColor(img_cv_resized, cv2.COLOR_BGR2GRAY)
plt.imshow(img_cv_resized, cmap='gray')

# Lets import our module
- We need the following
    - Path finding libs
        - Pathlib
    - image manipulation library
        - PIL
        - cv2
    - Plotting libs
        - seaborn
        - maplotlib
    - data manipulation libs
        - Sklearn

In [ ]:
# Path finder
from pathlib import Path
import os

# Image manipulation
import cv2
from PIL import Image
import numpy as np

# plotting libs
import seaborn as sns
from matplotlib import pyplot as plt

# Data manipulation
from sklearn.model_selection import train_test_split
from collections import Counter

In [ ]:
# Lets define our path here
data_path = Path('/kaggle/input/glass-bangle-defect-detection-classification')
labels_map = {
    'defect': 0,
    'good': 1,
    'broken': 2
}
images_list = list(data_path.glob('**/*.jpg'))
labels = [img_p.parent.stem for img_p in images_list]

# EDA (Explatory Data Analysis)
- We can explore our data here doing the following
    - Plot distribution of labels
    - Plot out some images to visualize

In [ ]:
label_counts = Counter(labels)
total_lbls = [label for label, _ in label_counts.items()]
total_count = [count for _, count in label_counts.items()]

plt.figure(figsize=(15, 10))
sns.barplot(x=total_lbls, y=total_count)

# Add annotations
for i, v in enumerate(total_count):
    plt.text(i, v + 1, str(v), ha='center', va='bottom')

# Set labels and title
plt.xlabel('Labels')
plt.ylabel('Label Count')
plt.title('Label Distribution')

In [ ]:
plt.figure(figsize=(15, 10))
index = 0
for sel_image in np.random.choice(images_list, 9):
    img = Image.open(str(sel_image))
    plt.subplot(3, 3, index + 1)
    plt.title(sel_image.parent.stem)
    plt.imshow(img)
    
    index += 1

plt.tight_layout()

# Lets split our dataset into the following groups
- train and test following the lables
- We use stratify to keep the distribution even

In [ ]:
# Lets split the dataset
train_images, test_images, train_lbls, test_lbls = train_test_split(
    images_list, labels, 
    test_size=0.2, 
    random_state=2023, 
    stratify=labels
)

In [ ]:
train_label_counts = Counter(train_lbls)
test_label_counts = Counter(test_lbls)

tst_count = []
tr_count = []
lbls = []
for tr_lbl, ts_lbl in zip(train_label_counts.items(), test_label_counts.items()):
    tr_count.append(tr_lbl[1])
    tst_count.append(ts_lbl[1])
    lbls.append(ts_lbl[0])
    
plt.figure(figsize=(15, 10))
plt.subplot(1, 2, 1)
plt.title('Train Dist Count')
sns.barplot(x=lbls, y=tr_count)

plt.subplot(1, 2, 2)
plt.title('Test Dist Count')
sns.barplot(x=lbls, y=tst_count)

# # Add annotations
# for i, v in enumerate(total_count):
#     plt.text(i, v + 1, str(v), ha='center', va='bottom')

# # Set labels and title
# plt.xlabel('Labels')
# plt.ylabel('Label Count')
# plt.title('Label Distribution')

# Lets apply our torch modules here
- Using torchvision as a module we can apply our image transformations here
- Lets mix our dataset class along with our transform class
    - We can apply this via the getitem function

In [ ]:
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader

In [ ]:
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

In [ ]:
transform(Image.open(train_images[0])).shape

In [ ]:
# Lets create a dataset class for bangle defect
class BangleDataset(Dataset):
    def __init__(self, images, labels, label_map, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        self.label_map = label_map
    
    def __len__(self):
        return len(self.images)
    
    # Handles the logic - processing logic
    def __getitem__(self, index):
        sel_img = self.images[index]
        
        # Image manipulation
        # Open as pil image
        img_pil = Image.open(sel_img).convert('RGB')    
        if self.transform is not None:
            img_pil = self.transform(img_pil)
            
        # Label manipulation
        sel_lbl = self.labels[index]
        sel_lbl = self.label_map[sel_lbl]
    
        return {
            'image': img_pil,
            'labels': sel_lbl
        }

In [ ]:
train_dataset = BangleDataset(train_images, train_lbls, labels_map, transform)
test_dataset = BangleDataset(test_images, test_lbls, labels_map, transform)
next(iter(test_dataset))['image'].shape

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True, num_workers=2)
next(iter(test_dataloader))['image'].shape

# Lets start by building our model
- We can use the nn module
- We will build our cnn with the following moudles
    - Convolutional 2d Layer
    - Linear Layer

In [ ]:
# Lets start by importing our modules
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class BangleClassifier(nn.Module):
    def __init__(self, lbls:int=3):
        super(BangleClassifier, self).__init__()
        # Feature Extractor
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, 2),
            nn.ReLU(),
            nn.MaxPool2d((2, 2))
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 2),
            nn.ReLU(),
            nn.MaxPool2d((2, 2))
        )
        
        # Flatten layer
        self.flatten = nn.Flatten()
        
        # Decision making
        self.fc = nn.Sequential(
            nn.Linear(193600, 256),
            nn.Linear(256, lbls),
        )
    
    def forward(self, img:torch.tensor):
        x = self.conv1(img)
        x = self.conv2(x)
        x_pre = self.flatten(x)
        return self.fc(x_pre)

In [ ]:
model = BangleClassifier()

In [ ]:
import random
from tqdm import tqdm

# Lets write our training loop
- Similar to yesterday we can use the following training loop style
    - Gather data
        - Move to GPU
    - Zero out gradients (Empty out your changes)
    - Run forward pass (Run prediction)
    - Run backward pass (update model)
    - Run evaluation metrics

In [ ]:
# Lets declare some components and hyper params
epochs = 20
lr = 1e-4

# Declare our loss func
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Use device check if gpu is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device);

In [ ]:
def training_step(batch):
    # Gather move our data to gpu
    image_tensor = batch['image'].to(device)
    label_tensor = batch['labels'].to(device)
    
    # Zero out the gradients
    model.zero_grad()
    
    # Run forward pass
    logits = model(image_tensor)
    
    # Run backward pass
    loss = criterion(logits, label_tensor)
    loss.backward()
    optimizer.step()
    
    return loss.item()

def eval_step(batch):
    # Gather move our data to gpu
    image_tensor = batch['image'].to(device)
    label_tensor = batch['labels'].to(device)
    
    # Zero out the gradients
    model.zero_grad()
    
    # Run forward pass
    with torch.no_grad():
        logits = model(image_tensor)
    
    # Run backward pass
    loss = criterion(logits, label_tensor)
    
    return loss.item()

In [ ]:
for i in tqdm(range(epochs)):
    # Set to train mode
    model.train()
    train_loss = 0
    
    # Get our batch and run training step
    for batch in train_dataloader:
        train_loss += training_step(batch)
        
    f_train_loss = train_loss / len(train_dataloader)
    print('Train Loss: {} for epoch: {}'.format(f_train_loss, i))
        
    eval_loss = 0
    model.eval()
    for batch in test_dataloader:
        eval_loss += eval_step(batch)
    
    f_eval_loss = eval_loss / len(test_dataloader)
    print('Eval Loss: {} for epoch: {}'.format(f_eval_loss, i))

In [ ]:
plt.figure(figsize=(15, 10))
index = 1
for sel_img in tqdm(random.sample(images_list, 16)):
    # Load image
    img_pil = Image.open(sel_img)
    gt_label = sel_img.parent.stem

    # Transform 
    img_tensor = transform(img_pil)
    img_tensor = img_tensor.unsqueeze(0)

    # Prediction
    with torch.no_grad():
        logits = model(img_tensor.to(device))
    
    # Map it back to a string
    labels = list(labels_map.keys())
    pred = F.softmax(logits, dim=1)
    
    pred_lbl = labels[pred[0].argmax().item()]
    pred_score = pred.max().item()
    
    plt.subplot(4, 4, index)
    plt.title('GT: {}, Pred: {}, Score: {}'.format(gt_label, pred_lbl, round(pred_score, 4) * 100))
    plt.imshow(img_pil)
    
    index += 1 

plt.tight_layout()